In [20]:
%load_ext autoreload
%autoreload 2
    
import os
import torch
import torch.nn as nn
import torch.nn.functional as F

from socket import gethostname
from prokbert.config_utils import *
from prokbert.sequtils import *
from prokbert. training_utils import *
from prokbert.prokbert_tokenizer import ProkBERTTokenizer
from prokbert.prok_datasets import IterableProkBERTPretrainingDataset
from prokbert.general_utils import *
from prokbert.ProkBERTDataCollator import *
from transformers import MegatronBertConfig, MegatronBertForMaskedLM, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
from transformers import DataCollatorForLanguageModeling
from transformers import BertConfig, BertForMaskedLM
from socket import gethostname
from torch.optim.lr_scheduler import CosineAnnealingLR
from transformers import Trainer, AdamW
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, matthews_corrcoef
import torch._dynamo

basedir = '/home/c_evolb/scratch/gitrepos/prokbert-paper'
basedir = '/home/ligeti/gitrepos/prokbert-paper'

sys.path.append(join(basedir, 'bin'))
from evaluate_training import *
from models import *


pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', 30)
#pd.set_option('display.width', 4000)
#pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', True)
from IPython.core.interactiveshell import InteractiveShell


InteractiveShell.ast_node_interactivity = "all"

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from prokbert.prokbert_tokenizer import ProkBERTTokenizer

tokenization_parameters = {'kmer' : 6,
                          'shift' : 1}
tokenizer = ProkBERTTokenizer(tokenization_params=tokenization_parameters)
segment='ATTTCTTTNNNGGGTTT'

tokenizer.encode(segment)



[2, 1020, 4068, 3972, 1, 1, 1, 1, 1, 1, 1, 1, 2756, 3]

In [3]:

promoter_traindb_dir = '/home/c_evolb/scratch/gitrepos/promoter/results'
promoter_traindb_dir = '/home/ligeti/gitrepos/promoter/results'

training_output_file = join(promoter_traindb_dir, 'promoter_training_data.tsv')
test_output_file = join(promoter_traindb_dir, 'promoter_test_data.tsv')
val_output_file = join(promoter_traindb_dir, 'promoter_val_data.tsv')

## Loading the input data
train_segmentdb = pd.read_csv(training_output_file, sep='\t')
test_segmentdb = pd.read_csv(test_output_file, sep='\t')
val_segmentdb = pd.read_csv(val_output_file, sep='\t')

